In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pandas as pd
import os

import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from unidecode import unidecode
import sys
import os





In [ ]:
#df= pd.read_excel(os.path.join('..', '27_08_24.xlsx'))

path= [os.path.join('..', 'data', x) for x in os.listdir(os.path.join('..', 'data'))]

df_0= pd.read_excel(path[0])
df_1= pd.read_excel(path[1])
df_2= pd.read_excel(path[2])
df_3= pd.read_excel(path[3])
df_4= pd.read_excel(path[4])




In [43]:
df= pd.concat([df_0, df_1,df_2, df_3,df_4])

In [ ]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df


,CONSOLIDADO(S),ESPECIFICAÇÃO CRIME,MEIO EMPREGADO DEAC,MUNICÍPIO(S),BAIRRO(S),ANO DO FATO,MÊS DO FATO,RISP,AISP,DATA DO FATO,...,IDADE VÍTIMA,FAIXA ETÁRIA VÍTIMA,TIPO DE VÍTIMA,COR VÍTIMA,GRAU DE INSTRUÇÃO VÍTIMA,ESTADO CIVÍL VÍTIMA,SEXO AUTOR,IDADE AUTOR,COR AUTOR,GRAU DE RELACIONAMENTO
0,HOMICIDIO,ACHADO DE CADAVER/EM DECOMPOSICAO,ARMA PERFUROCORTANTE,BELEM,TENONE,2024,MARCO,01ª RISP,13ª AISP,2024-03-13,...,64.0,ADULTO IV (35 A 64 ANOS),IDOSO(A),PARDA,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),P,66666.0,PREJUDICADO,NaN
1,LESAO CORPORAL,NaN,CONTUNDENTE,SANTAREM,LIBERDADE,2024,MAIO,12ª RISP,NaN,2024-05-22,...,50.0,ADULTO IV (35 A 64 ANOS),PREJUDICADO,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,66666.0,PREJUDICADO,NaN
2,LESAO CORPORAL,VIOLENCIA DOMESTICA,ARMA PERFUROCONTUNDENTE,BELEM,NAZARE,2024,MAIO,01ª RISP,02ª AISP,2024-05-03,...,21.0,ADULTO I (18 A 24 ANOS),PREJUDICADO,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,20.0,PREJUDICADO,COMPANHEIRO(A)
3,LESAO CORPORAL,MORDIDA DE CACHORRO,ARMA PERFUROCONTUNDENTE,BREVES,CENTRO DE BREVES,2024,MAIO,08ª RISP,NaN,2024-05-23,...,25.0,ADULTO II (25 A 29 ANOS),PREJUDICADO,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,CASADO(A),P,66666.0,PREJUDICADO,NaN
4,LESAO CORPORAL,VIOLENCIA DOMESTICA,VIOLENCIA FISICA,MELGACO,CENTRO DE MELGACO,2024,MAIO,08ª RISP,NaN,2024-05-11,...,43.0,ADULTO IV (35 A 64 ANOS),PREJUDICADO,PREJUDICADO,ENSINO MEDIO COMPLETO,SOLTEIRO(A),M,66666.0,PREJUDICADO,EX-COMPANHEIRO(A)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821566,ROUBO,TRANSEUNTE/CELULAR,VIOLENCIA,OURILANDIA DO NORTE,CENTRO DE OURILANDIA DO NORTE,2024,JANEIRO,14ª RISP,NaN,2024-01-01,...,19.0,ADULTO I (18 A 24 ANOS),PREJUDICADO,PREJUDICADO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),M,66666.0,PREJUDICADO,NaN
821567,ROUBO,TRANSEUNTE/CELULAR,ARMA DE FOGO,BELEM,MARAMBAIA,2024,JANEIRO,01ª RISP,09ª AISP,2024-01-01,...,32.0,ADULTO III (30 A 34 ANOS),PREJUDICADO,PREJUDICADO,ENSINO SUPERIOR COMPLETO,SOLTEIRO(A),M,66666.0,PARDA,PREJUDICADO
821568,LESAO CORPORAL,NaN,VIOLENCIA FISICA,MARITUBA,SAO FRANCISCO MARITUBA,2024,JANEIRO,02ª RISP,22ª AISP,2024-01-01,...,37.0,ADULTO IV (35 A 64 ANOS),PREJUDICADO,PREJUDICADO,ENSINO MEDIO COMPLETO,UNIAO ESTAVEL,F,66666.0,PREJUDICADO,COMPANHEIRO(A)
821569,FURTO,MEDIDOR DE ENERGIA,SEM INSTRUMENTO,MARABA,NOVA MARABA,2024,JANEIRO,10ª RISP,NaN,2024-01-01,...,88888.0,"PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS","PJ, EP E OUTROS",P,66666.0,PREJUDICADO,NaN


<div style= "background-color: gray; font-family: Segoe UI, Segoe UI Midlevel, Segoe WP, Arial, sans-serif; color: black;">Primeiramente, tive que copiar cada cidade da lista de cidades do estado do pará, pegando cada um por subregião (como: região metropolitana). Então, tive que transformar cada nome de cidade em letra maiuscula e tirar todos os acentos de cada nome de cidade. (Precisou ser dessa forma pois, no dataframe, o nome de cada cidade não possui nenhuma acentuação).
Então, depois de terminar cada região, tive que verificar se encaixava com a quantidade de cidades existentes somando todos os casos apresentados (deveria somar 144, supostamente o número de cidades do estado do Pará). Todavia, houve duas cidades que possuia no conjunto de dados, que não existiam formalmente nos escritos da prefeitura:</div>
<div style= "background-color: black; color: white;">

- Pau D Arco


- ALTAMIRA/CASTELO DOS SONHOS</div>
<div style= "background-color: gray; font-family: Segoe UI, Segoe UI Midlevel, Segoe WP, Arial, sans-serif;">
Sendo cada um destes existentes no conjunto de dados como cidades diferentes.
PS: 'Pau D Arco' é uma subregião pertecente a área da cidade de Santa Bárbara. Já 'ALTAMIRA/CASTELO DOS SONHOS' é uma subregião pertecente a área da cidade ALTAMIRA 
Então, o Pau D Arco foi adicionado a região metropolitana e ALTAMIRA/CASTELO DOS SONHOS foi para região sudoeste
<div>

In [ ]:
def get_subregion(municipio):
    if pd.isna(municipio): 
        return 'Desconhecida'
    elif municipio in baixo_amazonas:
        return 'Baixo Amazonas'
    elif municipio in marajo:
        return 'Marajó'
    elif municipio in metropolitana:
        return 'Metropolitana'
    elif municipio in sudoeste:
        return 'Sudoeste'
    elif municipio in nordeste:
        return 'Nordeste'
    elif municipio in sudeste:
        return 'Sudeste'
    else:
        return 'Desconhecida'

baixo_amazonas= ['Alenquer', 'Almeirim', 'Belterra', 'Curuá', 'Mojuí dos Campos', 'Faro', 'Juruti', 'Monte Alegre', 'Óbidos', 'Oriximiná', 'Prainha', 'Santarém','Terra Santa', 'PORTO DE MOZ', 'PLACAS']
baixo_amazonas= list(map(str.upper, baixo_amazonas))
baixo_amazonas= [unidecode(item) for item in baixo_amazonas]
##########################################################################
marajo= ['Cachoeira do Arari','Chaves','Muaná','Ponta de Pedras','Salvaterra','Santa Cruz do Arari','Soure','Afuá','Anajás','Breves','Curralinho','São Sebastião da Boa Vista','Bagre','Gurupá','Melgaço','Portel']
marajo= list(map(str.upper, marajo))
marajo= [unidecode(item) for item in marajo]
marajo

##########################################################################
metropolitana= ['Ananindeua','Barcarena','Belém', 'Benevides', 'Marituba','Santa Bárbara do Pará','Bujaru','Castanhal','Inhangapi','Santa Izabel do Pará','Santo Antônio do Tauá', 'PAU D ARCO']
metropolitana= list(map(str.upper, metropolitana))
metropolitana= [unidecode(item) for item in metropolitana]
metropolitana
##########################################################################
sudoeste= ['Altamira','Anapu','Brasil Novo','Medicilândia','Pacajá','Senador José Porfírio','Uruará','Vitória do Xingu','Aveiro','Itaituba','Jacareacanga','Novo Progresso','Rurópolis','Trairão', 'ALTAMIRA/CASTELO DOS SONHOS']
sudoeste= list(map(str.upper, sudoeste))
sudoeste=  [unidecode(item) for item in sudoeste]
sudoeste
##########################################################################
nordeste= ['Augusto Corrêa', 'Bonito', 'Bragança', 'Capanema', 'Igarapé-Açu', 'Nova Timboteua', 'Peixe-Boi', 'Primavera', 'Quatipuru', 'Santa Maria do Pará', 'Santarém Novo', 'São Francisco do Pará', 'Tracuateua', 'Abaetetuba', 'Baião', 'Cametá', 'Igarapé-Miri', 'Limoeiro do Ajuru', 'Mocajuba', 'Oeiras do Pará', 'Aurora do Pará', 'Cachoeira do Piriá', 'Capitão Poço', 'Garrafão do Norte', 'Ipixuna do Pará', 'Irituia', 'Mãe do Rio', 'Nova Esperança do Piriá', 'Ourém', 'Santa Luzia do Pará', 'São Domingos do Capim', 'São Miguel do Guamá', 'Viseu', 'Colares', 'Curuçá', 'Magalhães Barata', 'Maracanã', 'Marapanim', 'Salinópolis', 'São Caetano de Odivelas', 'São João da Ponta', 'São João de Pirabas', 'Terra Alta', 'Vigia', 'Acará', 'Concórdia do Pará', 'Moju', 'Tailândia', 'Tomé-Açu']
nordeste=list(map(str.upper, nordeste))
nordeste=  [unidecode(item) for item in nordeste]
##########################################################################
sudeste= ['Conceição do Araguaia', 'Floresta do Araguaia', 'Santa Maria das Barreiras', 'Santana do Araguaia', 'Brejo Grande do Araguaia', 'Marabá', 'Palestina do Pará', 'São Domingos do Araguaia', 'São João do Araguaia', 'Abel Figueiredo', 'Bom Jesus do Tocantins', 'Dom Eliseu', 'Goianésia do Pará', 'Paragominas', 'Rondon do Pará', 'Ulianópolis', 'Água Azul do Norte', 'Canaã dos Carajás', 'Curionópolis', 'Eldorado dos Carajás', 'Parauapebas', 'Pau d\'Arco', 'Piçarra', 'Redenção', 'Rio Maria', 'São Geraldo do Araguaia', 'Sapucaia', 'Xinguara', 'Bannach', 'Cumaru do Norte', 'Ourilândia do Norte', 'São Félix do Xingu', 'Tucumã', 'Breu Branco', 'Itupiranga', 'Jacundá', 'Nova Ipixuna', 'Novo Repartimento', 'Tucuruí']
sudeste=list(map(str.upper, sudeste))
sudeste=[unidecode(item) for item in sudeste]

# Adicionar a nova coluna 'Sub-Região' ao DataFrame
df['Sub-Região'] = df['MUNICÍPIO(S)'].apply(get_subregion)
df['Sub-Região'].unique()

In [101]:
df.columns

Index(['CONSOLIDADO(S)', 'ESPECIFICAÇÃO CRIME', 'MEIO EMPREGADO DEAC',
       'MUNICÍPIO(S)', 'BAIRRO(S)', 'ANO DO FATO', 'MÊS DO FATO', 'RISP',
       'AISP', 'DATA DO FATO', 'DIA DA SEMANA', 'HORA DO FATO',
       'FAIXA 4 HORAS', 'FAIXA 12 HORAS', 'LATITUDE', 'LONGITUDE', 'DISTRITO',
       'REGIÃO DE INTEGRAÇÃO', 'LOCAL DA OCORRÊNCIA', 'SEXO VÍTIMA',
       'IDADE VÍTIMA', 'FAIXA ETÁRIA VÍTIMA', 'TIPO DE VÍTIMA', 'COR VÍTIMA',
       'GRAU DE INSTRUÇÃO VÍTIMA', 'ESTADO CIVÍL VÍTIMA', 'SEXO AUTOR',
       'IDADE AUTOR', 'COR AUTOR', 'GRAU DE RELACIONAMENTO'],
      dtype='object')

In [45]:
lista_sexo= ['M', 'F', 'NÃO INFORMADO', 'PREJUDICADO', 'MASCULINO', 'FEMININO']
df['AISP'].fillna('PREJUDICADO', inplace=True)
df['GRAU DE RELACIONAMENTO'].fillna('PREJUDICADO', inplace=True)
df['IDADE AUTOR']=df['IDADE AUTOR'].apply(lambda x: x if x<100 else 'PREJUDICADO')

df['SEXO AUTOR']= df['SEXO AUTOR'].apply(lambda x: x if x in lista_sexo else 'PREJUDICADO')

#### A coluna "ESTADO CIVÍL VÍTIMA" é um ótimo exemplo de como um desenvolvimento de software pode impactar durante a análise de dados. 
#### Na coluna em questão, ao invés de existir apenas 5 tipos de estados civis oficiais (segundo a legislação Brasileira), as quais são : **SOLTEIRO(A), CASADO(A), DIVORCIADO(A), VIÚVO(A), SEPARADO(A)**, existem diversas outras entradas, algumas inválidas.
#### Junto a isso, dentre elas, estão presentes entradas com erros de digitação.
No código abaixo, observe o quantos tipos de input **solteiro** existem.

_ Solução: Ao invés de usar um Text Input para se preencher o campo de Estado Civíl (em casos de formulário online), colocar um dropdown button, por exemplo.

In [24]:
pd.DataFrame(sorted(df['ESTADO CIVÍL VÍTIMA'].dropna().unique()), columns={'Tipos Estado Civil':"0"}).iloc[19:29]

,Tipos Estado Civil
19,SOLTEIRA
20,SOLTEIRA(A)
21,SOLTEIRO
22,SOLTEIRO (A)
23,SOLTEIRO(A)
24,SOLTEIRO(A)_x000D_\n
25,SOLTERIO(A)
26,SOLTERIRO(A)
27,SOLTIRO(A)
28,SOTEIRO(A)


In [46]:

def CleanEstadoCivil(arg):
    if str(arg).upper()  in ['SOLTEIRO(A)_x000D_\n', "SOLTEIRO", "SOLTEIRO(A)_x000D_\n", "SOTEIRO(A)", "SOLTEIRO (A)", "SOLTEIRA(A)", "SOLTERIO(A)", "SOLTIRO(A)", "SOLTERIRO(A)", 'SOLTEIRA' ]:
        return 'SOLTEIRO(A)'
    elif str(arg).upper()    in ['CASADO', 'CASADO(A)']:
        return 'CASADO(A)'
    elif str(arg).upper()    in ['DIVORCIADO', 'DIVORCIADO(A)']:
        return 'DIVORCIADO(A)'
    elif str(arg).upper()    in ['VIUVO', 'VIUVO(A)', 'VIUVA', 'VIUVA(A)']:
        return 'VIUVO(A)'


    elif str(arg).upper()    in ['SEPARADO', 'SEPARADO(A)', 'SEPARADO(A)_x000D_']:
        return 'SEPARADO(A)'

    elif str(arg).upper()    in ['UNIAO ESTAVEL', 'UNIAO ESTAVEL ', 'UNIÃO ESTÁVEL', 'UNIAO ESTÁVEL', 'UNIÃO ESTAVEL', "UNIÃO ESTÁVEL_x000D_\n"]:
        return 'UNIAO ESTAVEL(A)'
    
    else:
        return 'PREJUDICADO'
df['ESTADO CIVÍL VÍTIMA']= df['ESTADO CIVÍL VÍTIMA'].apply(CleanEstadoCivil)
df['ESTADO CIVÍL VÍTIMA'].unique()

array(['PREJUDICADO', 'CASADO(A)', 'UNIAO ESTAVEL(A)', 'DIVORCIADO(A)',
       'VIUVO(A)', 'SOLTEIRO(A)', 'SEPARADO(A)'], dtype=object)

In [51]:
df['GRAU DE INSTRUÇÃO VÍTIMA'].unique()
def CleanGrauInstrucao(arg):
    if str(arg).upper()  in ['ENSINO FUNDAMENTAL COMPLETO', 'ENSINO FUNDAMENTAL COMPLETO_x000D_\n', 'ENSINO FUNDAMENTAL COMPLETO_x000D_', 'ENSINO FUNDAMENTAL COMPLETO_x000D_\n', 'ENSINO FUNDAMENTAL INCOMPLET']:
        return 'ENSINO FUNDAMENTAL COMPLETO'
    elif str(arg).upper()    in ['ENSINO FUNDAMENTAL INCOMPLETO', 'ENSINO FUNDAMENTAL INCOMPLETO_x000D_\n', 'ENSINO FUNDAMENTAL INCOMPLETO_x000D_', ' ENSINO FUNDAMENTAL INCOMPLETO']:
        return 'ENSINO FUNDAMENTAL INCOMPLETO'
    elif str(arg).upper()    in ['ENSINO MÉDIO COMPLETO', 'ENSINO MÉDIO COMPLETO_x000D_\n', 'ENSINO MÉDIO COMPLETO_x000D_', 'ENSINO MEDIO COMPLETO']:
        return 'ENSINO MÉDIO COMPLETO'
    elif str(arg).upper()    in ['ENSINO MÉDIO INCOMPLETO', 'ENSINO MÉDIO INCOMPLETO_x000D_\n', 'ENSINO MÉDIO INCOMPLETO_x000D_', 'ENSINO MEDIO INCOMPLETO']:
        return 'ENSINO MÉDIO INCOMPLETO'
    elif str(arg).upper()    in ['SUPERIOR COMPLETO', 'SUPERIOR COMPLETO_x000D_\n', 'SUPERIOR COMPLETO_x000D_', 'ENSINO SUPERIOR COMPLETO']:
        return 'SUPERIOR COMPLETO'
    elif str(arg).upper()    in ['SUPERIOR INCOMPLETO', 'SUPERIOR INCOMPLETO_x000D_\n', 'SUPERIOR INCOMPLETO_x000D_', 'ENSINO SUPERIOR INCOMPLETO']:
        return 'SUPERIOR INCOMPLETO'
    elif str(arg).upper()    in ['ANALFABETO', 'ANALFABETO_x000D_\n', 'ANALFABETO_x000D_', 'NAO ALFABETIZADO']:
        return 'ANALFABETO'
    else:
        return 'PREJUDICADO'
df['GRAU DE INSTRUÇÃO VÍTIMA']=df['GRAU DE INSTRUÇÃO VÍTIMA'].apply(CleanGrauInstrucao)
df['GRAU DE INSTRUÇÃO VÍTIMA'].unique()

array(['ENSINO FUNDAMENTAL INCOMPLETO', 'ENSINO MÉDIO COMPLETO',
       'PREJUDICADO', 'SUPERIOR COMPLETO', 'SUPERIOR INCOMPLETO',
       'ENSINO MÉDIO INCOMPLETO', 'ENSINO FUNDAMENTAL COMPLETO',
       'ANALFABETO'], dtype=object)

In [ ]:
df['COR VÍTIMA'].apply(lambda x: 'PREJUDICADO' if x=='PJ, EP E OUTROS' else x  ).unique()
####################################################################
####################################################################
####################################################################

####################################################################
####################################################################
####################################################################

array(['PARDA', 'PREJUDICADO', 'PRETA', 'BRANCA', 'INDIGENA', 'AMARELA',
       'JA LANCADO', nan], dtype=object)

In [ ]:
df['TIPO DE VÍTIMA'].fillna('PREJUDICADO', inplace=True)
df['TIPO DE VÍTIMA'].unique()
####################################################################
####################################################################
####################################################################
#   TESTAR ALGUM ALGORITMO DE LLM COMO SIMILARIDADE DE COSENO PARA UNIR PALAVRAS PARECIDAS
####################################################################
####################################################################
####################################################################


array(['IDOSO(A)', 'PREJUDICADO', 'PJ, EP E OUTROS', 'PM', 'MENOR',
       'DOPADO(A)', 'RECEM NASCIDO', 'ALCOOLIZADO(A)',
       'EX-PRESIDIARIO(A)/USUARIO(A) DE DROGAS',
       'ANTECEDENTES/USUARIO(A) DE DROGAS', 'PESSOA COM DEFICIENCIA',
       'PRESIDIARIO(A)', 'USUARIO(A) DE DROGAS', 'MILITAR DO EXERCITO',
       'FOI DOPADO(A)', 'ANTECEDENTES',
       'ANTECEDENTES/EX-PRESIDIARIO(A)/TRAFICANTE',
       'ANTECEDENTES/ASSALTANTE/TRAFICANTE', 'PRESIDIARIO(A)/TRAFICANTE',
       'MORADOR DE RUA', 'TRAFICANTE', 'GM', 'AG',
       'ANTECEDENTES/EX-PRESIDIARIO(A)/USUARIO(A) DE DROGAS',
       'MENOR/USUARIO(A) DE DROGAS', 'EX-PRESIDIARIO(A)', 'ASSALTANTE',
       'LGBT', 'PRESIDIARIO(A)/LGBT', 'PP',
       'ANTECEDENTES/ASSALTANTE/EX-PRESIDIARIO(A)/USUARIO(A) DE DROGAS/TRAFICANTE',
       'TRAFICANTE/USUARIO(A) DE DROGAS', 'ANTECEDENTES/TRAFICANTE',
       'ALCOOLIZADO(A)/MENOR',
       'ANTECEDENTES/USUARIO(A) DE DROGAS/MORADOR(A) DE RUA', 'PC',
       'ASSALTANTE/PRESIDIARIO(A)', 'GE

In [77]:
df['FAIXA ETÁRIA VÍTIMA'].apply(lambda x: 'PREJUDICADO' if x=='JA LANCADO' else x, ).unique()

array(['ADULTO IV (35 A 64 ANOS)', 'ADULTO I (18 A 24 ANOS)',
       'ADULTO II (25 A 29 ANOS)', 'PJ, EP E OUTROS',
       'ADULTO III (30 A 34 ANOS)', 'ADOLESCENTE (12 A 17 ANOS)',
       'CRIANCA (0 A 11 ANOS)', 'IDOSO (65 ANOS OU MAIS)', 'PREJUDICADO',
       '040.744.492-00'], dtype=object)

In [97]:
df['IDADE VÍTIMA'].apply(lambda x: 'PREJUDICADO' if x>140 else x ).fillna('PREJUDICADO').unique()

array([64.0, 50.0, 21.0, 25.0, 43.0, 'PREJUDICADO', 47.0, 31.0, 54.0,
       22.0, 35.0, 32.0, 39.0, 33.0, 30.0, 36.0, 18.0, 34.0, 38.0, 44.0,
       29.0, 15.0, 10.0, 65.0, 55.0, 45.0, 51.0, 24.0, 26.0, 77.0, 37.0,
       67.0, 70.0, 48.0, 23.0, 20.0, 40.0, 14.0, 83.0, 11.0, 12.0, 46.0,
       49.0, 78.0, 16.0, 52.0, 13.0, 17.0, 66.0, 58.0, 42.0, 28.0, 19.0,
       9.0, 27.0, 6.0, 53.0, 71.0, 0.0, 68.0, 80.0, 56.0, 60.0, 73.0, 4.0,
       57.0, 61.0, 63.0, 59.0, 75.0, 41.0, 74.0, 69.0, 76.0, 79.0, 84.0,
       62.0, 87.0, 81.0, 3.0, 5.0, 72.0, 99.0, 8.0, 88.0, 7.0, 2.0, 85.0,
       89.0, 91.0, 1.0, 82.0, 86.0, 92.0, 102.0, 90.0, 94.0, 93.0, 117.0,
       97.0, 107.0, 106.0, 95.0, 100.0, 119.0, 110.0, 103.0, 112.0, 116.0,
       96.0, 105.0, 101.0, 104.0, 111.0, 120.0, 98.0, 121.0, 108.0, 123.0],
      dtype=object)

In [99]:
df['RISP'].unique()

array(['01ª RISP', '12ª RISP', '08ª RISP', '02ª RISP', '10ª RISP',
       '04ª RISP', '16ª RISP', '03ª RISP', '13ª RISP', '11ª RISP',
       '05ª RISP', '15ª RISP', '06ª RISP', '14ª RISP', '09ª RISP',
       '07ª RISP'], dtype=object)

In [107]:

df.shape

(821571, 30)

In [10]:
#month_en=["JANUARY", "FEBRUARY", "MARCH", "APRIL", "MAY", "JUNE", "JULY", "AUGUST", "SEPTEMBER", "OCTOBER", "NOVEMBER", "DECEMBER"]

In [115]:

app = Dash(__name__)
dfteste= pd.DataFrame([df['CONSOLIDADO(S)'], df['MÊS DO FATO'], df['ANO DO FATO']]).T
#dfteste= dfteste.replace({'HOMICIDIO': 'MURDER'})
#dfteste= dfteste.replace({"JANEIRO":"JANUARY", "FEVEREIRO":"FEBRUARY", "MARÇO":"MARCH", "ABRIL":"APRIL", "MAIO":"MAY", "JUNHO":"JUNE", "JULHO":"JULY", "AGOSTO":"AUGUST", "SETEMBRO":"SEPTEMBER", "OUTUBRO":"OCTOBER", "NOVEMBRO":"NOVEMBER", "DEZEMBRO":"DECEMBER"})
#dfteste['YEAR']=  df['ANO DO FATO']

dfteste= dfteste.groupby(['CONSOLIDADO(S)', 'MÊS DO FATO', 'ANO DO FATO']).size().reset_index(name='Contagem')
# Layout da aplicação

app.layout = html.Div([
    html.H1("Gráfico de Ocorrências por Mês e Ano", style={'textAlign': 'center', 'backgroundColor': 'white', 'marginTop': '20px'}),
    
    # Dropdown para selecionar o tipo de crime
    dcc.Dropdown(
        id='tipo-crime-dropdown',
        options=[{'label': crime, 'value': crime} for crime in dfteste['CONSOLIDADO(S)'].unique()],
        value=dfteste['CONSOLIDADO(S)'].unique()[0],  # Valor inicial
        clearable=False
    ),
    # Componente para exibir o gráfico
    dcc.Graph(id='crime-graph'),

    # Anotação fora do gráfico
    html.Div("Fonte: Banco de Dados SEGUP", style={'textAlign': 'center', 'marginTop': '20px'})
])

# Callback para atualizar o gráfico com base na seleção do dropdown
@app.callback(
    Output('crime-graph', 'figure'),
    Input('tipo-crime-dropdown', 'value')
)
def update_graph(selected_crime):
    # Filtrar o DataFrame pelo tipo de crime selecionado
    df_filtered = dfteste.loc[dfteste['CONSOLIDADO(S)'] == selected_crime]

    # Ordenar os meses
    meses_ordenados = ["JANEIRO", "FEVEREIRO", "MARÇO", "ABRIL", "MAIO", "JUNHO", 
                    "JULHO", "AGOSTO", "SETEMBRO", "OUTUBRO", "NOVEMBRO", "DEZEMBRO"]
    df_filtered.loc[:, 'MÊS DO FATO'] = pd.Categorical(df_filtered['MÊS DO FATO'], categories=meses_ordenados, ordered=True)
    df_filtered = df_filtered.sort_values(['ANO DO FATO', 'MÊS DO FATO'])

    
    # Criar o gráfico de linhas
    fig = px.line(df_filtered, x='MÊS DO FATO', y='Contagem', color='ANO DO FATO', 
                  title=f'Ocorrências de {selected_crime} por Mês e Ano', markers=True)
    
    return fig

# Rodar a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_12824\2857687910.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_12824\2857687910.py:39: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



In [12]:
df_vulneravel= df[df['CONSOLIDADO(S)']=='ESTUPRO']
df_vulneravel= pd.concat([df_vulneravel, df[df['CONSOLIDADO(S)']=='ESTUPRO DE VULNERAVEL']]).dropna(subset='GRAU DE RELACIONAMENTO')

df_vulneravel= df_vulneravel.loc[df_vulneravel['GRAU DE RELACIONAMENTO']!='PREJUDICADO']
df_vulneravel= df_vulneravel.loc[df_vulneravel['FAIXA ETÁRIA VÍTIMA']!='PREJUDICADO']

fig = px.sunburst(df_vulneravel, path=['FAIXA ETÁRIA VÍTIMA', 'GRAU DE RELACIONAMENTO' ] , title='Casos de estupro ocorridos no estado do Pará por grau de relacionamento', width=900, height=900)
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Contagem de casos: %{value} <extra></extra>')
annotations = [
    dict(
        x=-0.1, 
        y=-0.08, 
        text="Os maiores números de casos são entre vítimas que tem menos de 17 anos<br>Os maiores números de casos são com pessoas próximas e conhecidas", 
        font=dict(size=15, color="black"),
        showarrow=False,
        bgcolor="lightgrey",  
        align='left'
    ),
]

fig.update_layout(annotations=annotations, title_font=dict(size=20) )

fig.show()


In [13]:

df_filtered = df[df['MUNICÍPIO(S)'].isin(['BELEM', 'ANANINDEUA'])]

df_grouped = df_filtered.groupby(['MUNICÍPIO(S)', 'BAIRRO(S)']).size().reset_index(name='count')

df_top3 = df_grouped.sort_values(['MUNICÍPIO(S)', 'count'], ascending=False).groupby('MUNICÍPIO(S)').head(3)

fig = px.bar(df_top3, x='MUNICÍPIO(S)', y='count', color='BAIRRO(S)', barmode='group',
            title='Top 3 Bairros com Mais Incidentes em Belém e Ananindeua')
fig.show()

In [14]:

month_mapping = {
    'JANEIRO': 1, 'FEVEREIRO': 2, 'MARCO': 3, 'ABRIL': 4, 'MAIO': 5, 'JUNHO': 6,
    'JULHO': 7, 'AGOSTO': 8, 'SETEMBRO': 9, 'OUTUBRO': 10, 'NOVEMBRO': 11, 'DEZEMBRO': 12
}
df_heat= df.copy()
df_heat['MÊS DO FATO'] = df_heat['MÊS DO FATO'].replace(month_mapping)

# Agregar dados por ano e mês
heatmap_data = df_heat.groupby(['ANO DO FATO', 'MÊS DO FATO']).size().unstack(fill_value=0).T

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Viridis',
    colorbar=dict(title='Número de Ocorrências'),
    text=heatmap_data.values,        
    texttemplate='%{text:.0f}',       
    textfont=dict(size=12, color='white')  
))

# Adicionar títulos e rótulos
fig.update_layout(
    title='Heatmap de Ocorrências por Ano e Mês',
    xaxis_title='Ano do Fato',
    yaxis_title='Mês do Fato',
    yaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=list(month_mapping.keys())),
    xaxis=dict(tickmode='linear'),
    autosize=False,
    width=800,
    height=600
)

# Mostrar o heatmap
fig.show()


Primeiramente, tive que copiar cada cidade da lista de cidades do estado do pará, pegando cada um por subregião (como: região metropolitana). Então, tive que transformar cada nome de cidade em letra maiuscula e tirar todos os acentos de cada nome de cidade. (Precisou ser dessa forma pois, no dataframe, o nome de cada cidade não possui nenhuma acentuação).
Então, depois de terminar cada região, tive que verificar se encaixava com a quantidade de cidades existentes somando todos os casos apresentados (deveria somar 144, supostamente o número de cidades do estado do Pará). Todavia, houve duas cidades que possuia no conjunto de dados, que não existiam formalmente nos escritos da prefeitura:
- Pau D Arco
- ALTAMIRA/CASTELO DOS SONHOS
Sendo cada um destes existentes no conjunto de dados como cidades diferentes.
PS: 'Pau D Arco' é uma subregião pertecente a área da cidade de Santa Bárbara. Já 'ALTAMIRA/CASTELO DOS SONHOS' é uma subregião pertecente a área da cidade ALTAMIRA 
Então, o Pau D Arco foi adicionado a região metropolitana e ALTAMIRA/CASTELO DOS SONHOS foi para região sudoeste

In [16]:
a= pd.DataFrame(df[df['MUNICÍPIO(S)'].isin(baixo_amazonas)]['MUNICÍPIO(S)'].unique())
b= pd.DataFrame(df[df['MUNICÍPIO(S)'].isin(marajo)]['MUNICÍPIO(S)'].unique())
c= pd.DataFrame(df[df['MUNICÍPIO(S)'].isin(metropolitana)]['MUNICÍPIO(S)'].unique())
d=pd.DataFrame(df[df['MUNICÍPIO(S)'].isin(sudoeste)]['MUNICÍPIO(S)'].unique())
e= pd.DataFrame(df[df['MUNICÍPIO(S)'].isin(nordeste)]['MUNICÍPIO(S)'].unique())
f= pd.DataFrame(df[df['MUNICÍPIO(S)'].isin(sudeste)]['MUNICÍPIO(S)'].unique())
regions= pd.concat([a, b,c , d, e, f])
regions.shape
#sorted(regions[regions[0].duplicated(False)][0].to_list())


(145, 1)

In [18]:
a

,0
0,SANTAREM
1,ALMEIRIM
2,ALENQUER
3,MONTE ALEGRE
4,CURUA
5,TERRA SANTA
6,ORIXIMINA
7,MOJUI DOS CAMPOS
8,PRAINHA
9,PORTO DE MOZ


In [19]:
df[~df['MUNICÍPIO(S)'].isin(regions[0])]['MUNICÍPIO(S)'].unique()

array([], dtype=object)